In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Parameters

In [ ]:
hier = False

res = 0.25  # dva

defaults = {'x': 0, 'y': 0, 'r': 2}

# Load catalogs

In [ ]:
tag = 'hier_' if hier else ''
unit_df = pd.read_csv(f'../../db/{tag}units.csv.gz')
unit_df['Subject'] = [v[:2] for v in unit_df['Session']]

In [ ]:
ar_df = pd.read_csv('../../db/bank_array_regions.csv')
ar_df['Subject'] = [v[:2] for v in ar_df['Session']]
ar_df = ar_df.groupby(['Subject', 'Array ID']).first()
unit_df['Region'] = ar_df.loc[pd.MultiIndex.from_frame(unit_df[['Subject', 'Array ID']]), 'Region'].values

In [ ]:
slatdf_path = 'summary/rf_fit.csv.gz'
alatdf_path = 'summary/rf_fit-array_level.csv.gz'

assert Path(slatdf_path).is_file()
assert Path(alatdf_path).is_file()

slat_df = pd.read_csv(slatdf_path)
if hier:
    slat_df['Name'] = [f'{v0}/{v1}' for v0, v1 in slat_df[['Level', 'Name']].values]

alat_df = pd.read_csv(alatdf_path)

# Impute values

### Apply resolution for estimates

In [ ]:
slat_df[['x', 'y', 'r']] = (slat_df[['x', 'y', 'r']]/res).round().astype(int) * res
alat_df[['x', 'y', 'r']] = (alat_df[['x', 'y', 'r']]/res).round().astype(int) * res

### Per unit, then channel, then array, within session

In [ ]:
assert 'Level' in slat_df.columns
unit_df[[['x', 'y', 'r']]] = np.nan
mn = np.isnan(unit_df['x'].values)

for level in ('Unit', 'Channel', 'Array'):
    m = slat_df['Level'] == level
    assert m.any()
    sllat_df = slat_df[m].groupby(['Session', 'Name']).agg({k: np.nanmedian for k in 'xyr'})
    level_col = {'Unit': 'Name', 'Array': 'Array ID'}.get(level, level)
    if hier:
        unit_index = pd.MultiIndex.from_arrays([
            unit_df['Session'],
            [f'{level}/{n}' if level!='Unit' else n for n in unit_df[level_col]]])
    else:
        unit_index = pd.MultiIndex.from_frame(
            unit_df[['Session', level_col]].astype(str))
    unit_index.names = ['Session', 'Name']
    vals = sllat_df[['x', 'y', 'r']].reindex(unit_index).values

    msl_= np.isfinite(vals[:,0])
    msl = mn & msl_
    print(f'{msl_.mean()*100:.1f}% (n = {msl_.sum()}) of units have {level}-level estimates')
    print(f'\t{msl.mean()*100:.1f}% (n = {msl.sum()}) have no previous estimates')
    unit_df.loc[msl, ['x', 'y', 'r']] = vals[msl]
    unit_df.loc[msl, 'Source'] = level

    mn = np.isnan(unit_df['x'].values)
    print(f'{mn.mean()*100:.1f}% (n = {mn.sum()} of units still lack an estimate')
    print()

### Per array, across sessions

In [ ]:
df_ = alat_df.set_index(['Subject', 'Name'])
assert not df_.index.has_duplicates
vals = df_[['x', 'y', 'r']].reindex(pd.MultiIndex.from_frame(unit_df[['Subject', 'Array ID']])).values
ma = np.isfinite(vals[:,0])
m = ma & np.isnan(unit_df['x'].values)
unit_df.loc[m, ['x', 'y', 'r']] = vals[m]
unit_df.loc[m, 'Source'] = 'Array, across sess'

print(f'{ma.mean()*100:.1f}% (n = {ma.sum()} of units have array-level estimates')
print(f'\t{m.mean()*100:.1f}% (n = {m.sum()}) have no previous estimates')

mn = np.isnan(unit_df['x'].values)
print(f'{mn.mean()*100:.1f}% (n = {mn.sum()} of units still lack an estimate')

### Fixed default

In [ ]:
mx = np.isnan(unit_df['x'].values)
for k in 'xyr':
    unit_df.loc[mx, k] = defaults[k]
unit_df.loc[mx, 'Source'] = 'Default'
print(f'{mx.mean()*100:.1f}% (n = {mx.sum()} of units used default estimates')

mn = np.isnan(unit_df['x'].values)
print(f'{mn.mean()*100:.1f}% (n = {mn.sum()} of units still lack an estimate')
assert not mn.any()

### Save results

In [ ]:
assert np.isfinite(unit_df['x']).all()
assert all(len(v) for v in unit_df['Source'])
df = unit_df[['Session', 'Name', 'x', 'y', 'r', 'Source']]
df.to_csv(f'../../db/per_{tag}unit_rf.csv.gz', index=False)

In [ ]:
print('Fraction of estimates from each source:')
(df.groupby(['Source']).count() / len(df) * 100).round(1)

### Summary plots

In [ ]:
if hier:
    m = unit_df['Source'] != 'Default'
    for level in unit_df['Level'].unique():
        fig, axs = plt.subplots(1, 3, figsize=(9, 2.5))
        for x, ax in zip('xyr', axs):
            sns.histplot(
                data=unit_df[m&(unit_df['Level']==level)],
                x=x, hue='Region', stat='density',
                common_norm=False, binwidth=res, element='poly',
                fill=False, ax=ax)
        axs[0].set_title(level)
else:
    fig, axs = plt.subplots(1, 3, figsize=(9, 2.5))
    for x, ax in zip('xyr', axs):
        sns.histplot(
            data=unit_df[unit_df['Source']!='Default'],
            x=x, hue='Region', stat='density',
            common_norm=False, binwidth=res, element='poly',
            fill=False, ax=ax)